In [1]:
import sys
import yaml
import torch
import logging
from pathlib import Path

# Add the path to the directory containing the omnicell package
# Assuming the omnicell package is in the parent directory of your notebook
sys.path.append('..')  # Adjust this path as needed

import yaml
import torch
import logging
from pathlib import Path
from omnicell.config.config import Config, ETLConfig, ModelConfig, DatasplitConfig, EvalConfig, EmbeddingConfig
from omnicell.data.loader import DataLoader
from omnicell.constants import PERT_KEY, GENE_EMBEDDING_KEY, CONTROL_PERT
from train import get_model

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configure paths
MODEL_CONFIG = ModelConfig.from_yaml("/orcd/data/omarabu/001/njwfish/omnicell/configs/models/scot/scot.yaml")
ETL_CONFIG = ETLConfig(name = "no_preprocessing", log1p = False, drop_unmatched_perts = True)
EMBEDDING_CONFIG = EmbeddingConfig(pert_embedding='GenePT')

SPLIT_CONFIG = DatasplitConfig.from_yaml("/orcd/data/omarabu/001/njwfish/omnicell/configs/splits/repogle_k562_essential_raw/random_splits/rs_accP_k562_ood_ss:ns_20_2_most_pert_0.1/split_0/split_config.yaml")
EVAL_CONFIG = EvalConfig.from_yaml("/orcd/data/omarabu/001/njwfish/omnicell/configs/splits/repogle_k562_essential_raw/random_splits/rs_accP_k562_ood_ss:ns_20_2_most_pert_0.1/split_0/eval_config.yaml")  # Set this if you want to run evaluations

# Load configurations
config = Config(model_config=MODEL_CONFIG,
                 etl_config=ETL_CONFIG, 
                 datasplit_config=SPLIT_CONFIG, 
                 eval_config=EVAL_CONFIG)




#Alternatively you can initialize the config objects manually as follows:
# etl_config = ETLConfig(name = XXX, log1p = False, drop_unmatched_perts = False, ...)
# model_config = ...
# embedding_config = ...
# datasplit_config = ...
# eval_config = ...
# config = Config(etl_config, model_config, datasplit_config, eval_config)

# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize data loader and load training data
loader = DataLoader(config)
adata, pert_rep_map = loader.get_training_data()

# Get dimensions and perturbation IDs
input_dim = adata.obsm['embedding'].shape[1]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pert_ids = adata.obs[PERT_KEY].unique()
gene_emb_dim = adata.varm[GENE_EMBEDDING_KEY].shape[1] if GENE_EMBEDDING_KEY in adata.varm else None

print(f"Data loaded:")
print(f"- Number of cells: {adata.shape[0]}")
print(f"- Input dimension: {input_dim}")
print(f"- Number of perturbations: {len(pert_ids)}")



2025-02-08 18:33:20,373 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/repogle_k562_essential_raw.yaml
2025-02-08 18:33:20,376 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/satija_IFNB_raw.yaml
2025-02-08 18:33:20,377 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/adamson_INCOMPLETE.yaml
2025-02-08 18:33:20,378 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/satija_IFNB_HVG.yaml
2025-02-08 18:33:20,379 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/kang.yaml
2025-02-08 18:33:20,381 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/essential_gene_knockouts_raw.yaml
2025-02-08 18:33:20,382 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/satija_IFNG_raw_INCOMPLET

Using device: cuda


2025-02-08 18:33:23,834 - INFO - Loaded unpreprocessed data, # of data points: 310385, # of genes: 8563.
2025-02-08 18:33:23,835 - INFO - Preprocessing training data
2025-02-08 18:33:23,837 - INFO - Using identity features for perturbations
2025-02-08 18:33:23,957 - INFO - Removing observations with perturbations not in the dataset as a column
2025-02-08 18:33:24,146 - INFO - Removed 189 perturbations that were not in the dataset columns and 0 perturbations that did not have an embedding for a total of 189 perturbations removed out of an initial 2058 perturbations
/orcd/data/omarabu/001/njwfish/omnicell/notebooks/../omnicell/data/loader.py:175: ImplicitModificationWarning: Setting element `.obsm['embedding']` of view, initializing view as actual.
  adata.obsm["embedding"] = adata.X.toarray().astype('float32')
2025-02-08 18:34:00,859 - INFO - Doing OOD split


Data loaded:
- Number of cells: 279630
- Input dimension: 8563
- Number of perturbations: 1850


In [2]:
model = get_model(config.model_config.name, config.model_config.parameters, loader, pert_rep_map, input_dim, device, pert_ids)

2025-02-08 18:34:08,452 - INFO - SCOT model selected


In [50]:
model = SCOT(adata, pert_rep_map, 128, 64, 64, 10)

In [3]:
import numpy as np
from omnicell.models.datamodules import get_dataloader

_, dl = get_dataloader(
    adata, 
    pert_ids=np.array(adata.obs[PERT_KEY].values), 
    offline=False, 
    pert_map=model.pert_embedding, 
    collate=None,
    batch_size=256
)

Creating source indices
Creating target indices
Creating pert indices
Creating source and target dicts
Strata probs [1.85915765e-05 1.85915765e-05 1.85915765e-05 ... 3.62163911e-03
 3.68856878e-03 7.42175735e-03]


In [4]:
model.max_epochs = 30

In [ ]:
model.train(adata, None, dl=dl)

/home/njwfish/miniconda3/envs/omnicell/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
2025-02-08 18:34:57,266 - INFO - Training for 30 epochs
Epoch [1/30]:  55%|█████▌    | 113/204 [00:05<00:03, 22.93it/s, loss=3.2231, best=inf, lr=2.00e-04]

In [74]:
import numpy as np

logger.info("Running evaluation")

# evaluate each pair of cells and perts
eval_dict = {}
for cell_id, pert_id, ctrl_data, gt_data in loader.get_eval_data():
    logger.debug(f"Making predictions for cell: {cell_id}, pert: {pert_id}")
    model.total_adata = loader.complete_eval_adata

    preds = model.make_predict(ctrl_data, pert_id, cell_id)
    eval_dict[(cell_id, pert_id)] = (ctrl_data.X.toarray(), gt_data.X.toarray(), preds)
    break
    
if not config.etl_config.log1p:
    for (cell, pert) in eval_dict:  
        ctrl_data, gt_data, pred_pert = eval_dict[(cell, pert)]
        # normalize to sum to 1
        ctrl_data = ctrl_data / ctrl_data.sum(axis=1).reshape(-1, 1) * 10_000
        gt_data = gt_data / gt_data.sum(axis=1).reshape(-1, 1) * 10_000
        pred_pert = pred_pert / pred_pert.sum(axis=1).reshape(-1, 1) * 10_000
        eval_dict[(cell, pert)] =  (np.log1p(ctrl_data), np.log1p(gt_data), np.log1p(pred_pert))


2025-02-08 12:06:17,852 - INFO - Running evaluation
2025-02-08 12:06:17,853 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/repogle_k562_essential_raw.yaml
2025-02-08 12:06:17,855 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/satija_IFNB_raw.yaml
2025-02-08 12:06:17,857 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/adamson_INCOMPLETE.yaml
2025-02-08 12:06:17,858 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/satija_IFNB_HVG.yaml
2025-02-08 12:06:17,859 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/kang.yaml
2025-02-08 12:06:17,861 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/essential_gene_knockouts_raw.yaml
2025-02-08 12:06:17,862 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/

AttributeError: 'numpy.ndarray' object has no attribute 'to'

In [30]:
import scanpy as sc
from omnicell.evaluation.utils import get_DEGs, get_eval, get_DEG_Coverage_Recall, get_DEGs_overlaps
pval_threshold = 0.05
log_fold_change_threshold = 0.0

results_dict = {}
for (cell, pert) in eval_dict:  
    ctrl_data, gt_data, pred_pert = eval_dict[(cell, pert)]

    pred_pert = sc.AnnData(X=pred_pert)
    true_pert = sc.AnnData(X=gt_data)
    control = sc.AnnData(X=ctrl_data)

    logger.debug(f"Getting ground Truth DEGs for {pert} and {cell}")
    true_DEGs_df = get_DEGs(control, true_pert)
    signif_true_DEG = true_DEGs_df[true_DEGs_df['pvals_adj'] < pval_threshold]

    logger.debug(f"Number of significant DEGS from ground truth: {signif_true_DEG.shape[0]}")

    logger.debug(f"Getting predicted DEGs for {pert} and {cell}")
    pred_DEGs_df = get_DEGs(control, pred_pert)


    logger.debug(f"Getting evaluation metrics for {pert} and {cell}")
    r2_and_mse = get_eval(control, true_pert, pred_pert, true_DEGs_df, [100,50,20], pval_threshold, log_fold_change_threshold)

    logger.debug(f"Getting DEG overlaps for {pert} and {cell}")
    DEGs_overlaps = get_DEGs_overlaps(true_DEGs_df, pred_DEGs_df, [100,50,20], pval_threshold, log_fold_change_threshold)

    results_dict[(cell, pert)] = (r2_and_mse, DEGs_overlaps)



/home/njwfish/miniconda3/envs/omnicell/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/njwfish/miniconda3/envs/omnicell/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [31]:
for (cell, pert) in results_dict:
    r2_and_mse, DEGs_overlaps = results_dict[(cell, pert)]
    print(f"Cell: {cell}, Pert: {pert}")
    # print(f"R2 and MSE: {r2_and_mse}")
    print(f"DEGs Overlaps: {DEGs_overlaps}")
    print("-"*100)

Cell: k562, Pert: RPL15
DEGs Overlaps: {'Overlap_in_top_2157_DEGs': 1544, 'Overlap_in_top_100_DEGs': 76, 'Overlap_in_top_50_DEGs': 33, 'Overlap_in_top_20_DEGs': 10, 'Jaccard': 0.3181607059916396}
----------------------------------------------------------------------------------------------------


In [32]:
r2_and_mse


{'all_genes_mean_sub_diff_R': np.float32(0.9717789),
 'all_genes_mean_sub_diff_R2': np.float32(0.9443543),
 'all_genes_mean_sub_diff_MSE': np.float32(0.0007809753),
 'all_genes_mean_fold_diff_R': np.float32(0.9901514),
 'all_genes_mean_fold_diff_R2': np.float32(0.9803998),
 'all_genes_mean_fold_diff_MSE': np.float32(0.10270075),
 'all_genes_mean_R': np.float32(0.99854505),
 'all_genes_mean_R2': np.float32(0.99709225),
 'all_genes_mean_MSE': np.float32(0.0007809753),
 'all_genes_var_R': np.float32(0.9586433),
 'all_genes_var_R2': np.float32(0.918997),
 'all_genes_var_MSE': np.float32(0.0007778878),
 'all_genes_corr_mtx_R': np.float64(0.1617623330193023),
 'all_genes_corr_mtx_R2': np.float64(0.026167052383847655),
 'all_genes_corr_mtx_MSE': np.float64(0.007415360278498834),
 'all_genes_cov_mtx_R': np.float64(0.21595909353670287),
 'all_genes_cov_mtx_R2': np.float64(0.046638330081194376),
 'all_genes_cov_mtx_MSE': np.float64(0.00021893967419830623),
 'Top_2157_DEGs_sub_diff_R': np.float32